In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import re
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [78]:
file_path_mainpage = 'D:/ssicsync/Webscrap/Webscrape Org Mainpage (Text).xlsx'
file_path_subpage = 'D:/ssicsync/Webscrap/Webscrape Org Subpage (Text).xlsx'
df_main = pd.read_excel(file_path_mainpage)
df_sub= pd.read_excel(file_path_subpage)

In [ ]:
ref = pd.read_csv('D:/ssicsync/Webscrap//List of 90 Coy and SSIC.csv')

In [111]:
ssic = pd.read_excel('D:/ssicsync/ssic2020-detailed-definitions.xlsx', skiprows=[0,1,2,3], usecols=[0,1,3,5])

d:\Anaconda\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


EDA

In [80]:
df_main
#df_main.shape[0]

,Company,Linkedin Page,Org Category,Org Website,Text Content,Org Website Content,Filtered Links,Extracted Text
0,ABR HOLDINGS LIMITED,https://www.linkedin.com/company/abr-holdings-...,Food & Beverages,http://www.abr.com.sg/,ABR Holdings Limited (“ABR”) began as the owne...,"<html lang=""en-US"" class=""has-wpcom-gifting-ba...",['https://abr.com.sg/about-abr/'],ABR Holdings Limited ...
1,ABUNDANCE INTERNATIONAL LIMITED,https://www.linkedin.com/company/life-in-abund...,Non-profit Organizations,https://lifeinabundance.org,Life In Abundance International (LIA) is an Af...,"<html lang=""en-US"" class=""js js_active vc_des...",[],Home Page - Life In Abundance International ...
2,ABUNDANTE LIMITED,https://www.linkedin.com/company/abundantesegu...,Insurance,http://www.abundanteseguros.com.br,Abundante Corretora de Seguros - Todos os Ramos,"<html lang=""pt""><head>\n \n <meta charset=""u...",[],Abundante Corretora de Seguros Abundante Segu...
3,ACCRELIST LTD,https://www.linkedin.com/company/accrelist-ai-...,IT Services and IT Consulting,http://www.weh.sg/,Our Business scope:\n1. -Aesthestics Platform ...,<html><head>\n<title>403 Forbidden</title>\n</...,[],403 Forbidden Forbidden You don't have perm...
4,ACESIAN PARTNERS LIMITED,https://www.linkedin.com/company/acesiansystem...,Information Technology & Services,http://www.acesiansystems.com,Created in 2016 as part of the longer-establis...,NaN,[],NaN
...,...,...,...,...,...,...,...,...
85,TELECHOICE INTERNATIONAL LIMITED,https://www.linkedin.com/company/telechoiceint...,Telecommunications,http://www.telechoice.com.sg,We are a regional diversified provider and ena...,"<html lang=""en"" class="" js flexbox flexboxlega...",['http://telechoice.listedcompany.com/profile....,TeleChoice International Limited Welcome to T...
86,TT INTERNATIONAL LIMITED,https://www.linkedin.com/company/tt-internatio...,NaN,http://tt-intl.com,NaN,"<html lang=""en""><head>\n \n <meta charset=""u...",[],Home TT International Ltd ...
87,UNITED OVERSEAS INSURANCE LIMITED,https://www.linkedin.com/company/united-overse...,Insurance,https://www.uoi.com.sg/,"Founded in 1971, United Overseas Insurance Lim...","<html><head>\n<script type=""text/javascript"" s...",['https://www.uoi.com.sg//about/corporate-deve...,United Overseas Insurance Limited (UOI) ...
88,VENTURE CORPORATION LIMITED,https://www.linkedin.com/company/venturecorpor...,"Appliances, Electrical, and Electronics Manufa...",https://www.venture.com.sg/,Venture (SGX: V03.SI) was formed in 1989 throu...,"<html lang=""en-US""><head>\n\t<meta charset=""UT...",['https://www.venture.com.sg/about/'],Venture Corporation Limited – Official Website...


In [81]:
df_sub
#df_sub.shape[0]

,Company,Filtered Links,Org Website Content,Extracted Text
0,ABR HOLDINGS LIMITED,http://www.abr.com.sg/,"<html lang=""en-US"" class=""has-wpcom-gifting-ba...",ABR Holdings Limited ...
1,ABR HOLDINGS LIMITED,https://abr.com.sg/about-abr/,"<html lang=""en-US"" class=""has-wpcom-gifting-ba...",ABR Holdings Limited ...
2,ABUNDANCE INTERNATIONAL LIMITED,https://lifeinabundance.org,"<html lang=""en-US"" class=""js js_active vc_des...",Home Page - Life In Abundance International ...
3,ABUNDANTE LIMITED,http://www.abundanteseguros.com.br,"<html lang=""pt""><head>\n \n <meta charset=""u...",Abundante Corretora de Seguros Abundante Segu...
4,ACCRELIST LTD,http://www.weh.sg/,<html><head>\n<title>403 Forbidden</title>\n</...,403 Forbidden Forbidden You don't have perm...
...,...,...,...,...
255,VICOM LTD,https://www.vicom.com.sg/About/Our-Vision-Miss...,"<html lang=""en-US""><head id=""Head""><meta conte...",VICOM About Key Management ...
256,VICOM LTD,https://www.vicom.com.sg/Financials/News-About-Us,"<html lang=""en-US""><head id=""Head""><meta conte...",VICOM About Key Management ...
257,VICOM LTD,https://www.vicom.com.sg/About/Corporate-Respo...,"<html lang=""en-US""><head id=""Head""><meta conte...",VICOM About Key Management ...
258,VICOM LTD,https://www.vicom.com.sg/About/Board-Of-Directors,"<html lang=""en-US""><head id=""Head""><meta conte...",VICOM About Key Management ...


In [82]:
print ("Duplicates in df_main extracted text:" ,df_main['Extracted Text'].duplicated().sum())
print("Duplicates in df_main company name:" , df_main['Company'].duplicated().sum())
df_main.shape[0]

Duplicates in df_main extracted text: 11
Duplicates in df_main company name: 0


90

In [83]:
print ("Duplicates in df_sub extracted text:" ,df_sub['Extracted Text'].duplicated().sum())
print("Duplicates in df_sub company name:" , df_sub['Company'].duplicated().sum())
df_sub.shape[0]

Duplicates in df_sub extracted text: 181
Duplicates in df_sub company name: 170


260

In [84]:
df_main = df_main.drop_duplicates(subset=['Extracted Text'])
df_main.shape[0]

79

In [85]:
df_sub = df_sub.drop_duplicates(subset=['Extracted Text'])
df_sub = df_sub.drop_duplicates(subset=['Company'])
df_sub.reset_index(drop=True, inplace=True)
#print(df_sub['Extracted Text'].duplicated().sum()) 
df_sub.shape[0]

79

In [73]:
#data = df_main[df_main['Extracted Text'].apply(lambda x: len(str(x)) >= 4)]
#data = data[~data['Extracted Text'].str.contains('403', case=False)]
#data = data[~data['Extracted Text'].str.contains('Nan', case=False)]

#df_sub = df_sub[df_sub['Extracted Text'].apply(lambda x: len(str(x)) >= 4)]
#df_sub = df_sub[~df_sub['Extracted Text'].str.contains('403', case=False)]
#df_sub = df_sub[~df_sub['Extracted Text'].str.contains('404 Not Found', case=False)]
#df_sub = df_sub[~df_sub['Extracted Text'].str.contains('Nan', case=False)]

In [100]:
#Standadise format (Mainpage)
df_main['Extracted Text'] = df_main['Extracted Text'].str.strip()
df_main['Extracted Text'] = df_main['Extracted Text'].str.replace('\r\n', ' ', regex=False)
df_main['Extracted Text'] = df_main['Extracted Text'].str.replace('\n', ' ', regex=False)
df_main['Extracted Text'] = df_main['Extracted Text'].str.lower()
df_main

C:\Users\Admin\AppData\Local\Temp\ipykernel_32424\1609062423.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_main['Extracted Text'] = df_main['Extracted Text'].str.strip()
C:\Users\Admin\AppData\Local\Temp\ipykernel_32424\1609062423.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_main['Extracted Text'] = df_main['Extracted Text'].str.replace('\r\n', ' ', regex=False)
C:\Users\Admin\AppData\Local\Temp\ipykernel_32424\1609062423.py:4: SettingWithCopyWarning: 
A value is trying to be set on a cop

,Company,Linkedin Page,Org Category,Org Website,Text Content,Org Website Content,Filtered Links,Extracted Text
0,ABR HOLDINGS LIMITED,https://www.linkedin.com/company/abr-holdings-...,Food & Beverages,http://www.abr.com.sg/,ABR Holdings Limited (“ABR”) began as the owne...,"<html lang=""en-US"" class=""has-wpcom-gifting-ba...",['https://abr.com.sg/about-abr/'],abr holdings limited ...
1,ABUNDANCE INTERNATIONAL LIMITED,https://www.linkedin.com/company/life-in-abund...,Non-profit Organizations,https://lifeinabundance.org,Life In Abundance International (LIA) is an Af...,"<html lang=""en-US"" class=""js js_active vc_des...",[],home page - life in abundance international ...
2,ABUNDANTE LIMITED,https://www.linkedin.com/company/abundantesegu...,Insurance,http://www.abundanteseguros.com.br,Abundante Corretora de Seguros - Todos os Ramos,"<html lang=""pt""><head>\n \n <meta charset=""u...",[],abundante corretora de seguros abundante segu...
3,ACCRELIST LTD,https://www.linkedin.com/company/accrelist-ai-...,IT Services and IT Consulting,http://www.weh.sg/,Our Business scope:\n1. -Aesthestics Platform ...,<html><head>\n<title>403 Forbidden</title>\n</...,[],403 forbidden forbidden you don't have perm...
4,ACESIAN PARTNERS LIMITED,https://www.linkedin.com/company/acesiansystem...,Information Technology & Services,http://www.acesiansystems.com,Created in 2016 as part of the longer-establis...,NaN,[],NaN
...,...,...,...,...,...,...,...,...
85,TELECHOICE INTERNATIONAL LIMITED,https://www.linkedin.com/company/telechoiceint...,Telecommunications,http://www.telechoice.com.sg,We are a regional diversified provider and ena...,"<html lang=""en"" class="" js flexbox flexboxlega...",['http://telechoice.listedcompany.com/profile....,telechoice international limited welcome to t...
86,TT INTERNATIONAL LIMITED,https://www.linkedin.com/company/tt-internatio...,NaN,http://tt-intl.com,NaN,"<html lang=""en""><head>\n \n <meta charset=""u...",[],home tt international ltd ...
87,UNITED OVERSEAS INSURANCE LIMITED,https://www.linkedin.com/company/united-overse...,Insurance,https://www.uoi.com.sg/,"Founded in 1971, United Overseas Insurance Lim...","<html><head>\n<script type=""text/javascript"" s...",['https://www.uoi.com.sg//about/corporate-deve...,united overseas insurance limited (uoi) ...
88,VENTURE CORPORATION LIMITED,https://www.linkedin.com/company/venturecorpor...,"Appliances, Electrical, and Electronics Manufa...",https://www.venture.com.sg/,Venture (SGX: V03.SI) was formed in 1989 throu...,"<html lang=""en-US""><head>\n\t<meta charset=""UT...",['https://www.venture.com.sg/about/'],venture corporation limited – official website...


In [89]:
#Standadise format (Sub page)
df_sub['Extracted Text'] = df_sub['Extracted Text'].str.strip()
df_sub['Extracted Text'] = df_sub['Extracted Text'].str.replace('\r\n', ' ', regex=False)
df_sub['Extracted Text'] = df_sub['Extracted Text'].str.replace('\n', ' ', regex=False)
df_sub['Extracted Text'] = df_sub['Extracted Text'].str.lower()
df_sub.head()

,Company,Filtered Links,Org Website Content,Extracted Text
0,ABR HOLDINGS LIMITED,http://www.abr.com.sg/,"<html lang=""en-US"" class=""has-wpcom-gifting-ba...",abr holdings limited ...
1,ABUNDANCE INTERNATIONAL LIMITED,https://lifeinabundance.org,"<html lang=""en-US"" class=""js js_active vc_des...",home page - life in abundance international ...
2,ABUNDANTE LIMITED,http://www.abundanteseguros.com.br,"<html lang=""pt""><head>\n \n <meta charset=""u...",abundante corretora de seguros abundante segu...
3,ACCRELIST LTD,http://www.weh.sg/,<html><head>\n<title>403 Forbidden</title>\n</...,403 forbidden forbidden you don't have perm...
4,ACESIAN PARTNERS LIMITED,http://www.acesiansystems.com,NaN,NaN


In [90]:
ref

,UEN,entity_name,ssic_code,ssic_code2,oth_desc,oth_desc2,gsearch
0,197803023H,ABR HOLDINGS LIMITED,47219,64202.0,RESTAURANTS AND MANUFACTURING OF ICE CREAM,NaN,https://www.google.com/search?q=ABR HOLDINGS L...
1,197501572K,ABUNDANCE INTERNATIONAL LIMITED,46649,18113.0,"MANUFACTURING, TRADING, STORAGE OF CHEMICALS A...",NaN,https://www.google.com/search?q=ABUNDANCE INTE...
2,197902587H,ABUNDANTE LIMITED,23940,41009.0,MANUFACTURE OF READY MIXED CONCRETE CONCRETE P...,NaN,https://www.google.com/search?q=ABUNDANTE LIMI...
3,201413466E,ACCRELIST MEDICAL AESTHETICS (SPC) PTE. LTD.,96022,86202.0,NaN,NaN,https://www.google.com/search?q=ACCRELIST MEDI...
4,199505699D,ACESIAN PARTNERS LIMITED,41009,25111.0,GENERAL WHOLESALE TRADE (INCLUDING GENERAL IMP...,MANUFACTURING OF VENTILATED CAGES AND EXHAUST ...,https://www.google.com/search?q=ACESIAN PARTNE...
...,...,...,...,...,...,...,...
84,199802072R,TELECHOICE INTERNATIONAL LIMITED,46523,46900.0,NaN,NaN,https://www.google.com/search?q=TELECHOICE INT...
85,198403771D,TT INTERNATIONAL LIMITED,46900,64202.0,TRADING IN CONSUMER ELECTRICAL & ELECTRONICS P...,INVESTMENT HOLDING,https://www.google.com/search?q=TT INTERNATION...
86,197100152R,UNITED OVERSEAS INSURANCE LIMITED,65124,65123.0,NaN,"IMPORT, EXPORT IS CLASSIFIED AS MARINE INSURANCE",https://www.google.com/search?q=UNITED OVERSEA...
87,198402886H,VENTURE CORPORATION LIMITED,26201,NaN,"PROVIDE MANUFACTURING, DESIGN, LOGISTIC SERVIC...",NaN,https://www.google.com/search?q=VENTURE CORPOR...


In [91]:
#standardise col name
ref.rename(columns = {'entity_name': 'Company'}, inplace = True)
ref = ref.sort_values(by = ['Company', 'ssic_code']).drop_duplicates(subset = 'Company', keep = 'first')
ref = ref[['Company', 'ssic_code']]

#Remove the "." behind to have more matches
ref['Company'] = ref['Company'].apply(lambda x: x[:-1] if x[-1] == '.' else x)

ref

,Company,ssic_code
0,ABR HOLDINGS LIMITED,47219
1,ABUNDANCE INTERNATIONAL LIMITED,46649
2,ABUNDANTE LIMITED,23940
3,ACCRELIST MEDICAL AESTHETICS (SPC) PTE. LTD,96022
4,ACESIAN PARTNERS LIMITED,41009
...,...,...
84,TELECHOICE INTERNATIONAL LIMITED,46523
85,TT INTERNATIONAL LIMITED,46900
86,UNITED OVERSEAS INSURANCE LIMITED,65124
87,VENTURE CORPORATION LIMITED,26201


In [99]:
# Perform the left join again
df_merged = pd.merge(df_main, ref, on='Company', how='left')
#df_merged=df_merged[df_merged['ssic_code'].notna()]

df_merged

,Company,Linkedin Page,Org Category,Org Website,Text Content,Org Website Content,Filtered Links,Extracted Text,ssic_code
0,ABR HOLDINGS LIMITED,https://www.linkedin.com/company/abr-holdings-...,Food & Beverages,http://www.abr.com.sg/,ABR Holdings Limited (“ABR”) began as the owne...,"<html lang=""en-US"" class=""has-wpcom-gifting-ba...",['https://abr.com.sg/about-abr/'],abr holdings limited ...,47219.0
1,ABUNDANCE INTERNATIONAL LIMITED,https://www.linkedin.com/company/life-in-abund...,Non-profit Organizations,https://lifeinabundance.org,Life In Abundance International (LIA) is an Af...,"<html lang=""en-US"" class=""js js_active vc_des...",[],home page - life in abundance international ...,46649.0
2,ABUNDANTE LIMITED,https://www.linkedin.com/company/abundantesegu...,Insurance,http://www.abundanteseguros.com.br,Abundante Corretora de Seguros - Todos os Ramos,"<html lang=""pt""><head>\n \n <meta charset=""u...",[],abundante corretora de seguros abundante segu...,23940.0
3,ACCRELIST LTD,https://www.linkedin.com/company/accrelist-ai-...,IT Services and IT Consulting,http://www.weh.sg/,Our Business scope:\n1. -Aesthestics Platform ...,<html><head>\n<title>403 Forbidden</title>\n</...,[],403 forbidden forbidden you don't have perm...,NaN
4,ACESIAN PARTNERS LIMITED,https://www.linkedin.com/company/acesiansystem...,Information Technology & Services,http://www.acesiansystems.com,Created in 2016 as part of the longer-establis...,NaN,[],NaN,41009.0
...,...,...,...,...,...,...,...,...,...
74,TELECHOICE INTERNATIONAL LIMITED,https://www.linkedin.com/company/telechoiceint...,Telecommunications,http://www.telechoice.com.sg,We are a regional diversified provider and ena...,"<html lang=""en"" class="" js flexbox flexboxlega...",['http://telechoice.listedcompany.com/profile....,telechoice international limited welcome to t...,46523.0
75,TT INTERNATIONAL LIMITED,https://www.linkedin.com/company/tt-internatio...,NaN,http://tt-intl.com,NaN,"<html lang=""en""><head>\n \n <meta charset=""u...",[],home tt international ltd ...,46900.0
76,UNITED OVERSEAS INSURANCE LIMITED,https://www.linkedin.com/company/united-overse...,Insurance,https://www.uoi.com.sg/,"Founded in 1971, United Overseas Insurance Lim...","<html><head>\n<script type=""text/javascript"" s...",['https://www.uoi.com.sg//about/corporate-deve...,united overseas insurance limited (uoi) ...,65124.0
77,VENTURE CORPORATION LIMITED,https://www.linkedin.com/company/venturecorpor...,"Appliances, Electrical, and Electronics Manufa...",https://www.venture.com.sg/,Venture (SGX: V03.SI) was formed in 1989 throu...,"<html lang=""en-US""><head>\n\t<meta charset=""UT...",['https://www.venture.com.sg/about/'],venture corporation limited – official website...,26201.0


In [105]:
df_main_processed = df_merged[['Company', 'ssic_code', 'Extracted Text','Org Category']][(df_merged.ssic_code.notnull()) & (df_merged['Extracted Text'].notnull())].reset_index(drop = True)
#Rename to same column name as Linked In file format
df_main_processed.rename(columns={'Extracted Text':'Text Content'}, inplace = True)
df_main_processed
df_main_processed.to_excel('Website_main_for_Modelling.xlsx')

SSIC

In [112]:
for column in ssic.columns:
    ssic.loc[((ssic[column] == '<Blank>') | (ssic[column].isnull())), column] = ''
ssic = ssic[~ssic['SSIC 2020'].str.match(r'^[A-Z]$')]

In [114]:
# Define a cleaning function
def clean_text(text):
    if isinstance(text, str):
        # Replace newline characters with space
        text = text.replace('\n', ' ')
        # Replace special characters (bullet points) with a consistent delimiter
        text = text.replace('•', '- ')
        # Strip leading and trailing whitespace
        text = text.strip()
        text= text.lower()
        # Replace multiple spaces with a single space
        text = ' '.join(text.split())
    return text

In [ ]:
# Apply the cleaning function to the 'Info' column
ssic['Detailed Definitions'] = ssic['Detailed Definitions'].apply(clean_text)
ssic['Examples of Activities Classified Under this Code'] = ssic['Examples of Activities Classified Under this Code'].apply(clean_text)

ssic['textonly'] = ssic['SSIC 2020 Title'] + '.' + ssic['Detailed Definitions'] + '.' + ssic['Examples of Activities Classified Under this Code']
ssic = ssic[['SSIC 2020', 'textonly']]
ssic.columns = ['ssic_code', 'Text Content']

In [118]:
ssic = ssic[ssic['Text Content'].notnull()]
ssic

,ssic_code,Text Content
1,01,AGRICULTURE AND RELATED SERVICE ACTIVITIES..
2,011,"GROWING OF CROPS, MARKET GARDENING AND HORTICU..."
3,0111,Growing of Food Crops (Non-Hydroponics)..
4,01111,Growing of leafy and fruit vegetables.this sub...
5,01112,Growing of mushrooms.this sub-class includes t...
...,...,...
7118,97002,Activities of households as employers of other...
7119,99010,Foreign embassies and trade representative off...
7120,99010,Foreign embassies and trade representative off...
7121,99020,Foreign armed forces.foreign armed force base.


Sub Page

In [121]:
df_sub.rename(columns={'Extracted Text':'Text Content'},inplace=True)
df_sub

,Company,Filtered Links,Org Website Content,Text Content
0,ABR HOLDINGS LIMITED,http://www.abr.com.sg/,"<html lang=""en-US"" class=""has-wpcom-gifting-ba...",abr holdings limited ...
1,ABUNDANCE INTERNATIONAL LIMITED,https://lifeinabundance.org,"<html lang=""en-US"" class=""js js_active vc_des...",home page - life in abundance international ...
2,ABUNDANTE LIMITED,http://www.abundanteseguros.com.br,"<html lang=""pt""><head>\n \n <meta charset=""u...",abundante corretora de seguros abundante segu...
3,ACCRELIST LTD,http://www.weh.sg/,<html><head>\n<title>403 Forbidden</title>\n</...,403 forbidden forbidden you don't have perm...
4,ACESIAN PARTNERS LIMITED,http://www.acesiansystems.com,NaN,NaN
...,...,...,...,...
74,TELECHOICE INTERNATIONAL LIMITED,http://www.telechoice.com.sg,"<html lang=""en"" class="" js flexbox flexboxlega...",telechoice international limited corporate pr...
75,TT INTERNATIONAL LIMITED,http://tt-intl.com,"<html lang=""en""><head>\n \n <meta charset=""u...",home tt international ltd ...
76,UNITED OVERSEAS INSURANCE LIMITED,https://www.uoi.com.sg/,"<html><head>\n<script type=""text/javascript"" s...",uoi corporate profile ...
77,VENTURE CORPORATION LIMITED,https://www.venture.com.sg/,"<html lang=""en-US""><head>\n\t<meta charset=""UT...",about – venture corporation limited ...


In [122]:
# Perform the left join again
df_merged_subpage = pd.merge(df_sub, ref, on='Company', how='left')
#df_merged=df_merged[df_merged['ssic_code'].notna()]

df_merged_subpage

,Company,Filtered Links,Org Website Content,Text Content,ssic_code
0,ABR HOLDINGS LIMITED,http://www.abr.com.sg/,"<html lang=""en-US"" class=""has-wpcom-gifting-ba...",abr holdings limited ...,47219.0
1,ABUNDANCE INTERNATIONAL LIMITED,https://lifeinabundance.org,"<html lang=""en-US"" class=""js js_active vc_des...",home page - life in abundance international ...,46649.0
2,ABUNDANTE LIMITED,http://www.abundanteseguros.com.br,"<html lang=""pt""><head>\n \n <meta charset=""u...",abundante corretora de seguros abundante segu...,23940.0
3,ACCRELIST LTD,http://www.weh.sg/,<html><head>\n<title>403 Forbidden</title>\n</...,403 forbidden forbidden you don't have perm...,NaN
4,ACESIAN PARTNERS LIMITED,http://www.acesiansystems.com,NaN,NaN,41009.0
...,...,...,...,...,...
74,TELECHOICE INTERNATIONAL LIMITED,http://www.telechoice.com.sg,"<html lang=""en"" class="" js flexbox flexboxlega...",telechoice international limited corporate pr...,46523.0
75,TT INTERNATIONAL LIMITED,http://tt-intl.com,"<html lang=""en""><head>\n \n <meta charset=""u...",home tt international ltd ...,46900.0
76,UNITED OVERSEAS INSURANCE LIMITED,https://www.uoi.com.sg/,"<html><head>\n<script type=""text/javascript"" s...",uoi corporate profile ...,65124.0
77,VENTURE CORPORATION LIMITED,https://www.venture.com.sg/,"<html lang=""en-US""><head>\n\t<meta charset=""UT...",about – venture corporation limited ...,26201.0


In [138]:
df_sub_processed = df_merged_subpage[['Company', 'ssic_code', 'Text Content']][(df_merged_subpage.ssic_code.notnull()) & (df_merged_subpage['Text Content'].notnull())].reset_index(drop = True)
#Rename to same column name as Linked In file format
df_sub_processed
df_sub_processed.to_excel('Website_Sub_for_Modelling.xlsx')

Combine Main+Sub Text content

In [ ]:

ssic['textonly'] = ssic['SSIC 2020 Title'] + '.' + ssic['Detailed Definitions'] + '.' + ssic['Examples of Activities Classified Under this Code']

In [141]:
df_main_conc=df_main_processed[['Company','Text Content','ssic_code']]
df_sub_conc=df_sub_processed[['Company','Text Content','ssic_code']]

In [142]:
combined_df = pd.concat([df_main_conc, df_sub_conc], axis=0)

In [143]:
combined_df = combined_df.groupby(['Company', 'ssic_code'], as_index=False).agg({'Text Content': ' '.join})

In [145]:
combined_df.to_excel('Website_Combined(Main&Sub)_for_Modelling.xlsx')